# Importing the Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Importing and Augmenting the Dataset

In [ ]:
train_datagen = ImageDataGenerator(
    brightness_range = [0.3, 0.9],
    shear_range = 0.2,
    horizontal_flip = True,
    rescale = 1./255,
validation_split = 0.2)

train_data = train_datagen.flow_from_directory(
    "/Gesture",
    target_size = (150, 150),
    color_mode = 'rgb',
    class_mode = 'categorical',
    batch_size = 32,
    shuffle = True,
    subset = 'training')
validation_data = train_datagen.flow_from_directory(
    "/Gesture",
    target_size = (150, 150),
    color_mode = 'rgb',
    class_mode = 'categorical',
    batch_size = 32,
    shuffle = True,
    subset = 'validation')

In [ ]:
test_datagen = ImageDataGenerator(
    rescale = 1./255)
test_data = test_datagen.flow_from_directory(
    "/Gesture",
    target_size = (150, 150),
    color_mode = 'rgb',
    class_mode = 'binary',
    batch_size = 32,
    shuffle = False) 

# Data Exploration

In [ ]:
def plotImages(images_arr):
    fig, axes = plt.subplots(2, 2, figsize = (20,20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
imgs, labels = next(train_data)
plotImages(imgs)
print(labels[:4])

# Fine-tuning a Pre-Trained Model (MobileNet)

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/tensorflow/keras-applications/mobilenet/mobilenet_1_0_224_tf_no_top.h5 \
    -O /tmp/mobilenet_1_0_224_tf_no_top.h5

In [ ]:
from tensorflow.keras.applications import MobileNet
local_weights_file = '/tmp/mobilenet_1_0_224_tf_no_top.h5'

In [ ]:
def create_pre_trained_model(local_weights_file):
    pre_trained_model = MobileNet(input_shape = (150, 150, 3), include_top = False, weights = None) 
    pre_trained_model.load_weights(local_weights_file)
    
    for layer in pre_trained_model.layers:
        layer.trainable = False

    return pre_trained_model

In [ ]:
pre_trained_model = create_pre_trained_model(local_weights_file)
pre_trained_model.summary()

In [ ]:
last_desired_layer = pre_trained_model.get_layer('conv_pw_13_relu')
last_output = last_desired_layer.output

In [ ]:
def create_final_model(pre_trained_model, last_output):
    x = tf.keras.layers.Flatten()(last_output)
    x = tf.keras.layers.Dense(100, activation = 'relu')(x)
    x = tf.keras.layers.Dense(4, activation = 'softmax')(x)  
    
    model = tf.keras.Model(inputs = pre_trained_model.input, outputs = x)
    model.compile(optimizer = "adam", loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    return model

In [ ]:
model = create_final_model(pre_trained_model, last_output)
history = model.fit(x = train_data, validation_data = validation_data, epochs = 5)

In [ ]:
sns.set_style("darkgrid")
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))

plt.plot(epochs, loss, 'ro--', label = 'Training Loss')
plt.plot(epochs, val_loss, 'bo--', label = 'Validation Loss')
plt.title('Loss Curve', fontweight = "bold")
plt.legend(loc = 0)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

In [ ]:
plt.plot(epochs, acc, 'ro--', label = 'Training Accuracy')
plt.plot(epochs, val_acc, 'bo--', label = 'Validation Accuracy')
plt.title('Accuracy', fontweight = "bold")
plt.legend(loc = 0)
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
y_true = test_data.labels
y_prob = model.predict(test_data)
y_pred = np.argmax(y_prob, axis = 1)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
sns.set_style("white")
c_m = confusion_matrix(y_true, y_pred)

labels = test_data.class_indices.keys()
display_c_m = ConfusionMatrixDisplay(c_m, display_labels = labels)
display_c_m.plot(cmap = 'Blues')
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
plt.title('Confusion Matrix', fontsize = 24)
plt.show()

In [ ]:
print(classification_report(y_true, y_pred))

# Testing Model on Single Image

In [ ]:
from tensorflow.keras.preprocessing import image
def prepare_image(file):
    img = image.load_img(file, target_size = (150, 150))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis = 0)
    return tf.keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)

In [ ]:
from IPython.display import Image
Image("Path to image file", width=500,height=500)

In [ ]:
preprocessed_image = prepare_image("Path to image file")
predictions = model.predict(preprocessed_image)
print(np.argmax(predictions))

# Saving Model Weights and Architecture

In [ ]:
model.save_weights('my_model_weights.h5')

In [ ]:
with open('my_model_arch.json', 'w') as f:
    json.dump(model.to_json(), f)